In [ ]:
# 필요한 라이브러리 로드
import os
import pandas as pd
import xlwings as xw

In [ ]:
# 'adobe_raw' 디렉토리에서 처리할 원본 파일 탐색
# 'New_Weekly_캠페인_방문_후_구매_Ken'으로 시작하는 파일 사용
for filename in os.listdir('./adobe_raw/'):
    if filename.startswith('New_Weekly_캠페인_방문_후_구매_Ken'):
        campaign_file = './adobe_raw/' + filename
        break

In [ ]:
# xlwings로 Excel 파일을 열고 데이터 읽기
# DRM 보호 CSV 파일을 읽기 위해 Excel 앱을 직접 제어하는 xlwings 사용
# try...finally 구문으로 Excel 앱의 확실한 종료 보장
workbook = None
try:
    workbook = xw.Book(campaign_file)
    sheet1 = workbook.sheets[0].used_range.value
    df_campaign = pd.DataFrame(sheet1)
finally:
    if workbook:
        workbook.app.quit()

In [ ]:
# 데이터프레임 초기 정제
df_campaign.columns = ['CampaignName', 'LastTouchChannel', 'Order', 'Revenue']
# 헤더 부분 제외 및 인덱스 재설정. .copy()로 SettingWithCopyWarning 방지
df_campaign = df_campaign.iloc[8:].reset_index(drop=True).copy()

In [ ]:
# 리포트 내 불필요한 데이터 블록 삭제
# '###' 구분선 사이의 짝수 번째 블록(캠페인 별 소계 등)을 삭제하는 로직
sharp_indices = df_campaign[df_campaign['CampaignName'] == '##############################################'].index

# sharp_indices를 2개씩 짝지어 그 사이의 범위(range)를 삭제
for i in range(0, len(sharp_indices), 2):
    start_index = sharp_indices[i]
    end_index = sharp_indices[i+1] + 1
    df_campaign.drop(labels=range(start_index, end_index), inplace=True)

In [ ]:
# 최종 데이터 필터링
# 결측치 및 컬럼명으로 잘못 인식된 행들 제거
df_campaign = df_campaign[
    df_campaign['LastTouchChannel'].notna() &
    (~df_campaign['LastTouchChannel'].isin(['Last Touch Channel', 'Marketing Channel']))
].copy()

In [ ]:
# 데이터 타입 변경
# 오류 방지를 위해 pd.to_numeric 사용
df_campaign['Order'] = pd.to_numeric(df_campaign['Order'], errors='coerce').fillna(0).astype(int)
df_campaign['Revenue'] = pd.to_numeric(df_campaign['Revenue'], errors='coerce').fillna(0).astype(int)

In [ ]:
# CampaignName 컬럼 값 정제
# 불필요한 문자열 제거
df_campaign['CampaignName'] = df_campaign['CampaignName'].str.replace("방문 후 구매","").str.replace("(dy)","").str.replace("(Ken)","").str.strip()

In [ ]:
# 중복 데이터 확인 (분석용)
df_campaign[df_campaign.duplicated(['CampaignName', 'LastTouchChannel'])]

In [ ]:
# CSV 파일로 저장
df_campaign.to_csv('./csv_raw/campaign_visit_then_order.csv', index=False, encoding='utf-8-sig')